### Contextualización del análisis
El presente notebook tiene como finalidad el análisis y la representación visual de distintos conjuntos de datos ambientales del Ayuntamiento de Madrid. Se abordan aspectos como la calidad del aire, los niveles de ruido y las condiciones meteorológicas, integrando visualizaciones interactivas mediante la librería Altair. El objetivo final es ofrecer una herramienta de exploración visual que facilite la comprensión de los fenómenos urbanos y su evolución en el tiempo.

In [ ]:
# Importación de librerías necesarias para la adquisición, procesamiento y visualización de datos ambientales.
import pandas as pd
import numpy as np
import requests
from io import StringIO
from datetime import datetime
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()
# Para quitar los warnings de altair
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import re
# Configuración general
pd.set_option('display.max_columns', None)

# Objetivos del proyecto

**Objetivo general**  
Analizar la evolución temporal y distribución espacial de los principales indicadores ambientales urbanos en la ciudad de Madrid durante el periodo 2022–2025, con el fin de detectar patrones, correlaciones y anomalías que puedan informar políticas públicas en materia de sostenibilidad, calidad del aire y salud ambiental. El análisis se implementará mediante visualizaciones interactivas que integran variables de calidad del aire, condiciones meteorológicas y contaminación acústica.

**Objetivos parciales**
1. Explorar la evolución temporal de los niveles de contaminación atmosférica (NO₂, PM10, O₃) por estación y por mes, e identificar tendencias, picos estacionales y posibles efectos de las condiciones meteorológicas sobre su concentración.
2. Visualizar las condiciones meteorológicas (temperatura, humedad, presión) en el mismo rango temporal, facilitando su análisis conjunto con los contaminantes atmosféricos para evaluar posibles relaciones.
3. Analizar los niveles diarios de contaminación acústica, incluyendo su comportamiento por franjas horarias (día, tarde, noche) y su posible correlación con otros indicadores ambientales y variables climáticas.
4. Proponer una representación espacial e interactiva de los datos, incorporando la localización de las estaciones de medición para permitir un análisis geográfico complementario.
5. Desarrollar visualizaciones interactivas con filtrado por variable, estación y periodo de tiempo, que permitan al usuario explorar de forma autónoma las relaciones entre los distintos indicadores ambientales.

## Descarga y procesamiento de datos desde el portal de datos abiertos de Madrid

In [ ]:
# Descarga y consolidación de datos diarios de calidad del aire correspondientes a Madrid, para los años 2022 a 2025.

urls = {
    "2025": "https://datos.madrid.es/egob/catalogo/201410-10306624-calidad-aire-diario.csv",
    "2024": "https://datos.madrid.es/egob/catalogo/201410-10306621-calidad-aire-diario.csv",
    "2023": "https://datos.madrid.es/egob/catalogo/201410-10306618-calidad-aire-diario.csv",
    "2022": "https://datos.madrid.es/egob/catalogo/201410-10306615-calidad-aire-diario.csv"
}

# Descargar y combinar los datasets de los años 2022 a 2025
dataframes = []
for year, url in urls.items():
    try:
        df = pd.read_csv(url, sep=';', encoding='latin1')
        df["AÑO"] = int(year)
        dataframes.append(df)
    except Exception as e:
        print(f"Error cargando {year}: {e}")

# Concatenar todos los dataframes
df_calidad_aire = pd.concat(dataframes, ignore_index=True)

print("\nTotal de registros leídos:", len(df_calidad_aire))

df_calidad_aire.head()  # Para ver las primeras filas de los datos cargados


In [ ]:

# Carga de información referenciada de estaciones de control de calidad del aire del municipio.
url_estaciones_calidad = "https://datos.madrid.es/egob/catalogo/212629-1-estaciones-control-aire.csv"
df_estaciones_aire = pd.read_csv(url_estaciones_calidad, sep=';', encoding="UTF-8")
df_estaciones_aire.rename(columns={'ESTACION': 'NOMBRE'}, inplace=True)

df_estaciones_aire.head()



In [ ]:
# Información general del dataframe
# Df_calidad_aire.info()
# Muestra aleatoria de 10 registros
df_calidad_aire.sample(10)

In [ ]:
# Transformar el dataframe al formato fila por dia para análisis (transponer para cada fila las oclumnas dxx y vxx)
import pandas as pd

# Columnas fijas
columnas_fijas = ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES']

# Generar lista de columnas de datos y validaciones
columnas_valor = [f'D{str(i).zfill(2)}' for i in range(1, 32)]
columnas_validez = [f'V{str(i).zfill(2)}' for i in range(1, 32)]

# Dataframe extendido para valores
df_valores = df_calidad_aire[columnas_fijas + columnas_valor].copy()
df_valores = df_valores.melt(id_vars=columnas_fijas, var_name='DIA', value_name='VALOR')

df_valores['DIA'] = df_valores['DIA'].str.extract(r'(\d+)').astype(int)

# Dataframe extendido para validez
df_validacion = df_calidad_aire[columnas_fijas + columnas_validez].copy()
df_validacion = df_validacion.melt(id_vars=columnas_fijas, var_name='DIA', value_name='VALIDO')

df_validacion['DIA'] = df_validacion['DIA'].str.extract(r'(\d+)').astype(int)

# Combinar ambos por columnas clave
df_calidad_largo = pd.merge(df_valores, df_validacion, on=columnas_fijas + ['DIA'])

# Añadir columna fecha combinada
df_calidad_largo['FECHA'] = pd.to_datetime(dict(year=df_calidad_largo['ANO'], month=df_calidad_largo['MES'], day=df_calidad_largo['DIA']), errors='coerce')

# Mostrar resultado
df_calidad_largo.head(10)

In [ ]:
print("\nTotal de registros antes de eliminar fecha:", len(df_calidad_largo))
# Ver si hay fechas imposibles
df_calidad_largo[df_calidad_largo['FECHA'].isna()].head(10)
# Eliminar fechas no válidas
df_calidad_largo = df_calidad_largo[df_calidad_largo['FECHA'].notna()]
print("\nTotal de registros :", len(df_calidad_largo))

In [ ]:
# Diccionario técnico de magnitudes (contaminantes y parámetros ambientales)
diccionario_magnitudes = {
    1: 'SO₂ - Dióxido de azufre',
    6: 'NO₂ - Dióxido de nitrógeno',
    7: 'PM10 - Partículas en suspensión',
    8: 'CO - Monóxido de carbono',
    9: 'O₃ - Ozono troposférico',
    10: 'Tolueno',
    12: 'Xileno',
    14: 'Benceno',
    20: 'Temperatura',
    30: 'HCT - Hidrocarburos totales',
    35: 'CH₄ - Metano',
    38: 'Radiación solar global',
    39: 'Velocidad del viento',
    70: 'Precipitación',
    81: 'Velocidad del viento (media)',
    82: 'Velocidad del viento (máxima)',
    83: 'Dirección del viento',
    84: 'Radiación solar máxima',
    85: 'Temperatura máxima',
    86: 'Humedad relativa',
    87: 'Presión atmosférica',
    88: 'Temperatura mínima',
    89: 'Temperatura media'
}


# Añadir columna con el nombre del contaminante
df_calidad_largo['MAGNITUD_NOMBRE'] = df_calidad_largo['MAGNITUD'].map(diccionario_magnitudes)
df_calidad_largo[['MAGNITUD', 'MAGNITUD_NOMBRE']].drop_duplicates().sort_values('MAGNITUD')
# Comporbar si hay alguna magnitud sin nombre
df_calidad_largo[df_calidad_largo['MAGNITUD_NOMBRE'].isna()].head(50)
print("\nTotal de registros con magnitud:", len(df_calidad_largo))
# Añadir el nombre de la estación
# Hacer el merge entre estacion de df_calidad_largo y código_corto de df_estaciones_aire
df_calidad_largo = df_calidad_largo.merge(
    df_estaciones_aire[['CODIGO_CORTO','NOMBRE']],
    left_on='ESTACION',
    right_on='CODIGO_CORTO',
    how='left'
)
# Renombrar a nombre_estacion
df_calidad_largo.rename(columns={'NOMBRE': 'NOMBRE_ESTACION'}, inplace=True)
# Eliminar la columna código_corto que queda duplicada
# If 'código_corto' in df_meteo_largo.columns:
df_calidad_largo.drop(columns=['CODIGO_CORTO'], inplace=True)
# Mostrar resultado
df_calidad_largo.head(10)

In [ ]:
# Preparar los archivos meteorológicos de 2022 a 2025 proceso similiar al de calidad

urls_meteo = {
    "2025": "https://datos.madrid.es/egob/catalogo/300351-21-meteorologicos-diarios.csv",
    "2024": "https://datos.madrid.es/egob/catalogo/300351-15-meteorologicos-diarios.csv",
    "2023": "https://datos.madrid.es/egob/catalogo/300351-12-meteorologicos-diarios.csv",
    "2022": "https://datos.madrid.es/egob/catalogo/300351-9-meteorologicos-diarios.csv"
}

dfs_meteo = []

# Descargar y guardar los datasets
for year, url in urls_meteo.items():
    try:
        df = pd.read_csv(url, sep=';', encoding='latin1')
        dfs_meteo.append(df)
    except Exception as e:
        print(f"Error al descargar o cargar los datos de {year}: {e}")

# Concatenar todos los años en un solo dataframe
df_meteo_total = pd.concat(dfs_meteo, ignore_index=True)

# Mostrar diez entradas aleatorias
df_meteo_total.sample(10)


In [ ]:
# Obtener datos de las estaciones  de meteorologia
url_estaciones_meteo = "https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv"
df_estaciones_meteo = pd.read_csv(url_estaciones_meteo, sep=';', encoding="UTF-8")
df_estaciones_meteo.rename(columns={'ESTACION': 'NOMBRE'}, inplace=True)

df_estaciones_meteo.head()

Obtener los datos de los nombres de las estaciones

In [ ]:
# Transformar df_meteo_total al formato entrada por dia, igual que calidad para análisis diario
columnas_fijas = ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES']
columnas_valor = [f'D{str(i).zfill(2)}' for i in range(1, 32)]
columnas_validez = [f'V{str(i).zfill(2)}' for i in range(1, 32)]

# Crear tabla de valores
df_valores = df_meteo_total[columnas_fijas + columnas_valor].copy().melt(
    id_vars=columnas_fijas, var_name='DIA', value_name='VALOR')
df_valores['DIA'] = df_valores['DIA'].str.extract(r'(\d+)').astype(int)

# Crear tabla de validaciones
df_validacion = df_meteo_total[columnas_fijas + columnas_validez].copy().melt(
    id_vars=columnas_fijas, var_name='DIA', value_name='VALIDO')
df_validacion['DIA'] = df_validacion['DIA'].str.extract(r'(\d+)').astype(int)

# Unir valores y validaciones
df_meteo_largo = pd.merge(df_valores, df_validacion, on=columnas_fijas + ['DIA'])

# Crear columna de fecha y eliminar fechas inválidas
df_meteo_largo['FECHA'] = pd.to_datetime(dict(year=df_meteo_largo['ANO'], month=df_meteo_largo['MES'], day=df_meteo_largo['DIA']), errors='coerce')
df_meteo_largo = df_meteo_largo[df_meteo_largo['FECHA'].notna()]

# Añadir el nombre de la estación
# Hacemos el merge entre estacion de df_meteo_largo y código_corto de df_estaciones_meteo


df_meteo_largo = df_meteo_largo.merge(
    df_estaciones_meteo[['CÓDIGO_CORTO','NOMBRE']],
    left_on='ESTACION',
    right_on='CÓDIGO_CORTO',
    how='left'
)
# Renombrar a nombre_estacion
df_meteo_largo.rename(columns={'NOMBRE' : 'NOMBRE_ESTACION'}, inplace=True)
# Eliminar la columna código_corto que queda duplicada
# If 'código_corto' in df_meteo_largo.columns:
df_meteo_largo.drop(columns=['CÓDIGO_CORTO'], inplace=True)


# Vista rápida de 10 valores aleatorios
df_meteo_largo.sample(10)

In [ ]:
url_estaciones_ruido = "https://datos.madrid.es/egob/catalogo/211346-1-estaciones-acusticas.csv"
df_estaciones_ruido = pd.read_csv(url_estaciones_ruido, sep=';', encoding="latin-1")


# Mostrar una vista previa
df_estaciones_ruido.head()

In [ ]:
# Procesamiento de ruido ambiental
# Descargar el fichero con los datos
import pandas as pd

url = "https://datos.madrid.es/egob/catalogo/215885-10749127-contaminacion-ruido.csv"
df_ruido = pd.read_csv(url, sep=';', encoding='latin1')
# Extraer número de estación y convertir a entero
df_estaciones_ruido['ID_ESTACION'] = df_estaciones_ruido['ESTACIÓN'].str.extract(r'RF-(\d+)').astype(int)

# Crear diccionario de mapeo de nombre
mapa_estaciones_ruido = df_estaciones_ruido.set_index('ID_ESTACION')['NOMBRE'].to_dict()

# Añadir el nombre de la es
df_ruido['NOMBRE_ESTACION'] = df_ruido['NMT'].map(mapa_estaciones_ruido)

df_ruido.head(10)

In [ ]:
# Procesamiento del fichero de ruido

# Convertir columnas numéricas: reemplazar comas por puntos
columnas_numericas = ['LAeq', 'L1', 'L10', 'L50', 'L90', 'L99']
for col in columnas_numericas:
    df_ruido[col] = df_ruido[col].astype(str).str.replace(',', '.').astype(float)

# Filtrar años 2022 a 2025
df_ruido = df_ruido[df_ruido['Año'].between(2022, 2025)]

# Crear columna fecha
df_ruido['FECHA'] = pd.to_datetime({
    'year': df_ruido['Año'],
    'month': df_ruido['mes'],
    'day': df_ruido['dia']
}, errors='coerce')

df_ruido.info()
print(df_ruido.columns.tolist())
df_ruido[['Año', 'mes', 'dia']].isnull().sum()
# Mostrar los datos procesados
df_ruido.head()

In [ ]:
# Exportación de los conjuntos de datos a csv para presentación intermedia
# Exportar los tres conjuntos de datos a ficheros csv

# Exportar df_ruido a csv
df_ruido.to_csv('datos_ruido.csv', index=False)

# Exportar df_calidad_largo a csv
df_calidad_largo.to_csv('datos_calidad.csv', index=False)

# Exportar df_meteo_largo a csv
df_meteo_largo.to_csv('datos_meteo.csv', index=False)

print("Los tres conjuntos de datos han sido exportados correctamente a ficheros CSV.")

### Interpretación del gráfico de NO₂
Este gráfico permite observar la evolución del dióxido de nitrógeno (NO₂) en distintas estaciones de la ciudad de Madrid durante el periodo 2022–2025. 

**PM10** hace referencia a partículas en suspensión con un diámetro aerodinámico menor o igual a 10 micras. Estas partículas pueden proceder de fuentes naturales (polvo, polen) o antropogénicas (tráfico, industria, calefacción urbana). Debido a su pequeño tamaño, pueden penetrar en el sistema respiratorio y generar impactos negativos en la salud, especialmente en personas con enfermedades respiratorias o cardiovasculares.  

En la visualización que sigue se analiza la evolución de los niveles de PM10 registrados por las estaciones de medición de la ciudad de Madrid entre 2022 y 2025. El gráfico permite seleccionar una o varias estaciones para focalizar el análisis.
Se aprecia cómo algunas estaciones muestran niveles más elevados que otras de manera consistente, lo que puede reflejar zonas con mayor tráfico rodado o menor ventilación atmosférica. La estacionalidad también es visible, con picos recurrentes durante los meses fríos, en línea con lo esperado por el uso de calefacción y condiciones atmosféricas más estables.

In [ ]:
# Visualización de la evolución de pm10 (magnitud == 7) por estación con altair

# Filtrar datos para pm10 (magnitud == 7) y fechas válidas

df_pm10 = df_calidad_largo[(df_calidad_largo['MAGNITUD'] == 7) & (df_calidad_largo['VALIDO'] == 'V')].copy()
df_pm10 = df_pm10.sort_values('FECHA')

# Crear selección múltiple por estación
selector = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')

# Gráfico con control de opacidad según selección
chart_pm10 = alt.Chart(df_pm10).mark_line().encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(selector, alt.value(1), alt.value(0.1))
).add_params(
    selector
).properties(
    title='Evolución de PM10 por estación (2022–2025)',
    width=800,
    height=400
).interactive()

chart_pm10.save("chart_pm10.html")  # Exportación como HTML interactivo
chart_pm10

In [ ]:
# Filtrar pm10
df_pm10 = df_calidad_largo[(df_calidad_largo['MAGNITUD'] == 7) & (df_calidad_largo['VALIDO'] == 'V')].copy()
df_pm10 = df_pm10.sort_values('FECHA')

# Selecciones
estacion_sel_pm10 = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')
fecha_sel_pm10 = alt.selection_interval(encodings=['x'])

# Detalle
detalle_pm10 = alt.Chart(df_pm10).mark_line().encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(estacion_sel_pm10, alt.value(1), alt.value(0.05))
).transform_filter(
    fecha_sel_pm10
).add_params(
    estacion_sel_pm10
).properties(
    width=800,
    height=400
)

# Selector inferior
selector_pm10 = alt.Chart(df_pm10).mark_area(opacity=0.3).encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N'
).properties(
    width=800,
    height=60
).add_params(
    fecha_sel_pm10
)

chart_pm10 = detalle_pm10 & selector_pm10

chart_pm10.save("chart_pm10.html")  # Exportación como HTML interactivo
detalle_pm10.save("detalle_pm10.html")  # Exportación como HTML interactivo

chart_pm10

### Representación de NO₂

**NO₂** (dióxido de nitrógeno) es un contaminante atmosférico típico de entornos urbanos, producido principalmente por la combustión de carburantes fósiles, especialmente por el tráfico rodado y sistemas de calefacción. Es un contaminante regulado por normativa europea y nacional debido a sus efectos adversos sobre la salud respiratoria y cardiovascular.

En esta visualización se muestra la evolución temporal de los niveles diarios de NO₂ medidos en distintas estaciones de Madrid entre 2022 y 2025. El gráfico permite interactuar filtrando por estación y seleccionar un rango de fechas para focalizar el análisis. Esto permite identificar patrones estacionales, episodios de concentración elevada, y diferencias geográficas en la exposición.

In [ ]:

# Filtrar no₂
df_no2 = df_calidad_largo[(df_calidad_largo['MAGNITUD'] == 6) & (df_calidad_largo['VALIDO'] == 'V')].copy()
df_no2 = df_no2.sort_values('FECHA')

# Selección por estación (interactiva con la leyenda)
estacion_sel = alt.selection_point(fields=['ESTACION'], bind='legend')

# Selección por rango de fechas (intervalo en eje x)
fecha_sel = alt.selection_interval(encodings=['x'])

# Gráfico detalle con filtro por fecha y estación
detalle = alt.Chart(df_no2).mark_line().encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(estacion_sel, alt.value(1), alt.value(0.05))
).transform_filter(
    fecha_sel
).add_params(
    estacion_sel
).properties(
    width=800,
    height=400
)

# Gráfico resumen inferior (selector de rango de fechas)
selector = alt.Chart(df_no2).mark_area(opacity=0.3).encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N'
).properties(
    width=800,
    height=60
).add_params(
    fecha_sel
)

# Composición final
chart_no2 = detalle & selector

chart_no2.save("chart_no2.html")  # Exportación como HTML interactivo
detalle.save("detalle_no2.html")  # Exportación como HTML interactivo
chart_no2


### Representación de O₃ (Ozono troposférico)

**El ozono troposférico (O₃)** no se emite directamente, sino que se forma en la atmósfera a partir de otros contaminantes (como NO₂ y compuestos orgánicos volátiles) en presencia de luz solar. Es un contaminante secundario, y sus niveles suelen aumentar en los meses cálidos y en situaciones de alta radiación solar.

El O₃ es irritante para los pulmones y puede agravar enfermedades respiratorias. Esta visualización permite examinar su evolución temporal por estación y filtrar el periodo de análisis.

In [ ]:
# Filtrar o₃
df_o3 = df_calidad_largo[(df_calidad_largo['MAGNITUD'] == 9) & (df_calidad_largo['VALIDO'] == 'V')].copy()
df_o3 = df_o3.sort_values('FECHA')

# Selecciones
estacion_sel_o3 = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')
fecha_sel_o3 = alt.selection_interval(encodings=['x'])

# Detalle
detalle_o3 = alt.Chart(df_o3).mark_line().encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(estacion_sel_o3, alt.value(1), alt.value(0.05))
).transform_filter(
    fecha_sel_o3
).add_params(
    estacion_sel_o3
).properties(
    width=800,
    height=400
)

# Selector inferior
selector_o3 = alt.Chart(df_o3).mark_area(opacity=0.3).encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N'
).properties(
    width=800,
    height=60
).add_params(
    fecha_sel_o3
)

chart_o3 = detalle_o3 & selector_o3

detalle_o3.save("detalle_o3.html")  # Exportación como HTML interactivo
chart_o3


### Representación de Temperatura Media

La temperatura ambiental es una de las variables más relevantes para el análisis climático y ambiental. Su evolución permite entender patrones estacionales y su relación con otros factores como la concentración de contaminantes o el confort acústico urbano.
Este gráfico muestra los valores diarios de temperatura media registrados por las estaciones meteorológicas de Madrid entre 2022 y 2025.

In [ ]:
# Visualización: temperatura media (magnitud == 89)
df_temp = df_meteo_largo[(df_meteo_largo['MAGNITUD'] == 89) & (df_meteo_largo['VALIDO'] == 'V')].copy()
df_temp = df_temp.sort_values('FECHA')
sel_temp = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')
brush_temp = alt.selection_interval(encodings=['x'])
detalle_temp = alt.Chart(df_temp).mark_line().encode(
    x='FECHA:T', y='VALOR:Q', color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(sel_temp, alt.value(1), alt.value(0.05))
).transform_filter(brush_temp).add_params(sel_temp).properties(width=800, height=400)
selector_temp = alt.Chart(df_temp).mark_area(opacity=0.3).encode(
    x='FECHA:T', y='VALOR:Q', color='NOMBRE_ESTACION:N'
).add_params(brush_temp).properties(width=800, height=60)
chart_temp = detalle_temp & selector_temp


detalle_temp.save("detalle_temp.html")  # Exportación como HTML interactivo
chart_temp


### Representación de Humedad Relativa

La humedad relativa es un factor clave en la percepción térmica, la formación de contaminantes secundarios y la dispersión de partículas en el aire. Este gráfico permite analizar cómo ha evolucionado esta variable meteorológica en las distintas estaciones de medición.

In [ ]:
# Visualización: humedad relativa (magnitud == 86)
df_hum = df_meteo_largo[(df_meteo_largo['MAGNITUD'] == 86) & (df_meteo_largo['VALIDO'] == 'V')].copy()
df_hum = df_hum.sort_values('FECHA')
sel_hum = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')
brush_hum = alt.selection_interval(encodings=['x'])
detalle_hum = alt.Chart(df_hum).mark_line().encode(
    x='FECHA:T', y='VALOR:Q', color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(sel_hum, alt.value(1), alt.value(0.05))
).transform_filter(brush_hum).add_params(sel_hum).properties(width=800, height=400)
selector_hum = alt.Chart(df_hum).mark_area(opacity=0.3).encode(
    x='FECHA:T', y='VALOR:Q', color='NOMBRE_ESTACION:N'
).add_params(brush_hum).properties(width=800, height=60)
chart_hum = detalle_hum & selector_hum

detalle_hum.save("detalle_hum.html")  # Exportación como HTML interactivo
chart_hum


### Representación de Presión Atmosférica

La presión atmosférica puede influir en la dispersión vertical de los contaminantes, afectando a su concentración a nivel del suelo. También está relacionada con condiciones meteorológicas como estabilidad o tormentas. Esta visualización permite examinar su evolución temporal en diferentes estaciones de medición.

In [ ]:
# Visualización: presión atmosférica (magnitud == 87)
df_pres = df_meteo_largo[(df_meteo_largo['MAGNITUD'] == 87) & (df_meteo_largo['VALIDO'] == 'V')].copy()
df_pres = df_pres.sort_values('FECHA')
sel_pres = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')
brush_pres = alt.selection_interval(encodings=['x'])
detalle_pres = alt.Chart(df_pres).mark_line().encode(
    x='FECHA:T', y=alt.Y('VALOR:Q', scale=alt.Scale(domain=[900, 1000])), color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'VALOR:Q'],
    opacity=alt.condition(sel_pres, alt.value(1), alt.value(0.05))
).transform_filter(brush_pres).add_params(sel_pres).properties(width=800, height=400)
selector_pres = alt.Chart(df_pres).mark_area(opacity=0.3).encode(
    x='FECHA:T', y='VALOR:Q', color='NOMBRE_ESTACION:N'
).add_params(brush_pres).properties(width=800, height=60)
chart_pres = detalle_pres & selector_pres

detalle_pres.save("detalle_pres.html")  # Exportación como HTML interactivo
chart_pres


### Representación de Contaminación Acústica (Ruido)

Los niveles de ruido ambiental son un factor clave en la calidad de vida urbana y están regulados por normativa europea y local. En esta visualización se representa la evolución de distintas medidas acústicas registradas en Madrid entre 2022 y 2025, incluyendo:

- **LAeq**: nivel continuo equivalente (promedio energético)
- **L1, L10, L50, L90, L99**: percentiles que representan el nivel de presión sonora superado el 1%, 10%, 50%, 90% y 99% del tiempo, respectivamente

El gráfico permite seleccionar la estación (NMT), la variable de interés, y filtrar temporalmente. Solo se representa el dato total diario (tipo 'T').

In [ ]:
# Transformar df_ruido al formato largo y filtrar tipo 't'
df_ruido_filtrado = df_ruido[df_ruido['tipo'] == 'T'].copy()
df_ruido_largo = df_ruido_filtrado.melt(
    id_vars=['FECHA', 'NOMBRE_ESTACION', 'tipo'],
    value_vars=['LAeq', 'L1', 'L10', 'L50', 'L90', 'L99'],
    var_name='MEDIDA',
    value_name='VALOR'
)
df_ruido_largo = df_ruido_largo.sort_values('FECHA')

# Selecciones interactivas
sel_nmt = alt.selection_point(fields=['NOMBRE_ESTACION'], bind='legend')
sel_medida = alt.selection_point(
    fields=['MEDIDA'],
    bind=alt.binding_select(options=['LAeq', 'L1', 'L10', 'L50', 'L90', 'L99']),
    name='Variable'
)
brush_ruido = alt.selection_interval(encodings=['x'])

# Gráfico principal
detalle_ruido = alt.Chart(df_ruido_largo).mark_line().encode(
    x='FECHA:T',
    y='VALOR:Q',
    color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'MEDIDA:N', 'VALOR:Q'],
    opacity=alt.condition(sel_nmt, alt.value(1), alt.value(0.05))
).transform_filter(
    sel_medida
).transform_filter(
    brush_ruido
).add_params(
    sel_nmt, sel_medida
).properties(width=800, height=400)

# Gráfico selector temporal
selector_ruido = alt.Chart(df_ruido_largo).mark_area(opacity=0.3).encode(
    x='FECHA:T', y='VALOR:Q', color='NOMBRE_ESTACION:N'
).transform_filter(sel_medida).add_params(brush_ruido).properties(width=800, height=60)

chart_ruido = detalle_ruido & selector_ruido

detalle_ruido.save("detalle_ruido.html")  # Exportación como HTML interactivo
chart_ruido


### Análisis cruzado: NO₂ vs Temperatura Media

El dióxido de nitrógeno (NO₂) es un contaminante que tiende a acumularse más en condiciones frías y estables, mientras que las temperaturas elevadas favorecen su dispersión. En esta visualización se cruzan los datos de concentración diaria de NO₂ con la temperatura media diaria registrada en la misma estación y fecha.

Este tipo de análisis ayuda a identificar patrones de relación entre contaminantes y condiciones meteorológicas.

In [ ]:
# Preparar datos cruzados: no₂ vs temperatura media
no2 = df_calidad_largo[(df_calidad_largo['MAGNITUD'] == 6) & (df_calidad_largo['VALIDO'] == 'V')][
    ['FECHA', 'NOMBRE_ESTACION', 'VALOR']
].rename(columns={'VALOR': 'NO2'})

temp = df_meteo_largo[(df_meteo_largo['MAGNITUD'] == 89) & (df_meteo_largo['VALIDO'] == 'V')][
    ['FECHA', 'NOMBRE_ESTACION', 'VALOR']
].rename(columns={'VALOR': 'TEMP_MEDIA'})

# Unir por fecha y estacion
df_merge = pd.merge(no2, temp, on=['FECHA', 'NOMBRE_ESTACION'])

# Crear gráfico de dispersión
scatter = alt.Chart(df_merge).mark_circle(opacity=0.6, size=60).encode(
    x=alt.X('TEMP_MEDIA:Q', title='Temperatura Media (°C)'),
    y=alt.Y('NO2:Q', title='NO₂ (µg/m³)'),
    color='NOMBRE_ESTACION:N',
    tooltip=['FECHA:T', 'NOMBRE_ESTACION:N', 'TEMP_MEDIA:Q', 'NO2:Q']
).properties(
    width=700, height=400,
    title='Relación entre NO₂ y Temperatura Media (2022–2025)'
).interactive()

scatter.save("scatter.html")  # Exportación como HTML interactivo
scatter


### Localización de estaciones y condiciones ambientales
Mapa georreferenciado con la ubicación de estaciones y una codificación visual por magnitud seleccionada. Permite comprender la distribución espacial de los fenómenos registrados.
#### intento de poner un mapa geográfico de fondo en una capa

In [ ]:
# Crear df_map a partir de df_calidad_largo y df_estaciones
df_map = df_calidad_largo[['ESTACION', 'FECHA', 'MAGNITUD', 'VALOR', 'NOMBRE_ESTACION', 'ANO', 'MES', 'DIA']].copy()
df_map = df_map.merge(df_estaciones_aire, left_on='ESTACION', right_on='CODIGO_CORTO', how='left')
df_map['MAGNITUD_NOMBRE'] = df_map['MAGNITUD'].map(diccionario_magnitudes)
df_map = df_map.dropna(subset=['LATITUD', 'LONGITUD', 'MAGNITUD_NOMBRE'])
df_map.head()
# Selectores separados para año, mes y día el selector por fecha no funcionaba
selector_magnitud = alt.selection_point(
    fields=['MAGNITUD_NOMBRE'],
    bind=alt.binding_select(
        options=sorted(df_map['MAGNITUD_NOMBRE'].dropna().unique()),
        name='Magnitud: '
    )
)

selector_ano = alt.selection_point(
    fields=['ANO'],
    bind=alt.binding_select(
        options=sorted(df_map['ANO'].dropna().unique()),
        name='Año: '
    )
)

selector_mes = alt.selection_point(
    fields=['MES'],
    bind=alt.binding_select(
        options=sorted(df_map['MES'].dropna().unique()),
        name='Mes: '
    )
)

selector_dia = alt.selection_point(
    fields=['DIA'],
    bind=alt.binding_select(
        options=sorted(df_map['DIA'].dropna().unique()),
        name='Día: '
    )
)

# Gráfico base con los nuevos selectores
mapa_base = alt.Chart(df_map).mark_circle().encode(
    longitude='LONGITUD:Q',
    latitude='LATITUD:Q',
    size=alt.Size('VALOR:Q', scale=alt.Scale(range=[100, 800]), legend=None),
    color=alt.Color('VALOR:Q', 
                   scale=alt.Scale(scheme='redyellowgreen', reverse=True), 
                   legend=alt.Legend(title="Valor")),
    tooltip=['NOMBRE_ESTACION:N', 'ANO:Q', 'MES:Q', 'DIA:Q', 'MAGNITUD_NOMBRE:N', 'VALOR:Q']
).transform_filter(
    selector_magnitud
).transform_filter(
    selector_ano
).transform_filter(
    selector_mes
).transform_filter(
    selector_dia
).add_params(
    selector_magnitud, selector_ano, selector_mes, selector_dia
).project(
    type='mercator'
).properties(
    width=600,
    height=500,
    title='Mapa de valores ambientales por estación'
)

# Datos del mapa base
data_world = alt.topo_feature(data.world_110m.url, "countries")

# Crear el mapa base con los datos geoespaciales
extent = {
    "type": "Feature",
    "geometry": {"type": "Polygon",
                 "coordinates": [[
                     [-3.55, 40.52],  # [xmax, ymax] - Noreste
                     [-3.55, 40.33],  # [xmax, ymin] - Sureste
                     [-3.83, 40.33],  # [xmin, ymin] - Suroeste
                     [-3.83, 40.52],  # [xmin, ymax] - Noroeste
                     [-3.55, 40.52]   # [xmax, ymax] - Cierre del polígono
                 ]]
                },
    "properties": {}
}

base = (
    alt.Chart(data_world)
    .mark_geoshape(clip=True, fill="lightgray", stroke="black", strokeWidth=0.5)
    .project(type="mercator", fit=extent)
)




mapa_base.save("mapa_base.html")  # Exportación como HTML interactivo

# Combinar el mapa base con los puntos de datos
base + mapa_base
# Pdte o se pone un png de fondo o hay que conectar a un servicio de mapas

### Perfil climático diario (Temperatura, Humedad y Presión)

Visualización radial de tres variables meteorológicas normalizadas: temperatura media, humedad relativa y presión atmosférica. Permite comparar perfiles diarios y observar posibles desequilibrios entre las condiciones climáticas medidas.


In [ ]:
# Variables meteorológicas relevantes
clima_vars = {89: 'Temperatura media', 86: 'Humedad relativa', 87: 'Presión atmosférica'}
df_clima = df_meteo_largo[df_meteo_largo['MAGNITUD'].isin(clima_vars)].copy()
df_clima['VARIABLE'] = df_clima['MAGNITUD'].map(clima_vars)

# Filtrar solo combinaciones con las 3 magnitudes
grupo_clima = df_clima.groupby(['FECHA', 'ESTACION'])['VARIABLE'].nunique().reset_index()
grupo_clima = grupo_clima[grupo_clima['VARIABLE'] == 3]
df_clima = df_clima.merge(grupo_clima[['FECHA', 'ESTACION']], on=['FECHA', 'ESTACION'])

# Normalizar los valores por variable
df_clima['VALOR_ESCALADO'] = df_clima.groupby('VARIABLE')['VALOR'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
# Selectores interactivos mejorados
selector_ano = alt.selection_point(
    fields=['ANO'],
    bind=alt.binding_select(options=sorted(df_clima['ANO'].dropna().unique()), name="Año"),
    name='Año'
)
selector_mes = alt.selection_point(
    fields=['MES'],
    bind=alt.binding_select(options=sorted(df_clima['MES'].dropna().unique()), name="Mes"),
    name='Mes'
)
selector_dia = alt.selection_point(
    fields=['DIA'],
    bind=alt.binding_select(options=sorted(df_clima['DIA'].dropna().unique()), name="Día"),
    name='Día'
)

selector_estacion = alt.selection_point(
    fields=['NOMBRE_ESTACION'],
    bind=alt.binding_select(options=sorted(df_clima['NOMBRE_ESTACION'].dropna().unique()), name="Estación"),
    name='Estación'
)

# El resto del código del gráfico permanece igual
grafico_clima = (alt.Chart(df_clima)
           .transform_filter(selector_ano)
           .transform_filter(selector_mes)
           .transform_filter(selector_dia)
           .transform_filter(selector_estacion)
           .mark_arc(innerRadius=50)
           .encode(
               theta=alt.Theta('VARIABLE:N', sort=list(clima_vars.values())),
               radius='VALOR_ESCALADO:Q',
               color='VARIABLE:N',
               tooltip=['VARIABLE:N', 'VALOR:Q', alt.Tooltip('VALOR_ESCALADO:Q', title='Escalado')]
           ).add_params(
               selector_ano, selector_mes, selector_dia, selector_estacion
           ).properties(
               width=400,
               height=400,
               title='Perfil climático diario (normalizado)'
           ))



grafico_clima.save("grafico_clima.html")  # Exportación como HTML interactivo
grafico_clima


In [ ]:
### Perfil diario de calidad del aire (NO₂, PM10, O₃)

Este gráfico radial representa la composición relativa de contaminantes en un día concreto y en una estación determinada. Se normalizan los valores de NO₂, PM10 y O₃ para mostrar el reparto proporcional entre contaminantes. Resulta útil para detectar días atípicos con predominancia de uno de los contaminantes.


In [ ]:
# Variables de calidad del aire
aire_vars = {6: 'NO₂', 7: 'PM10', 9: 'O₃'}
df_aire = df_calidad_largo[df_calidad_largo['MAGNITUD'].isin(aire_vars)].copy()
df_aire['VARIABLE'] = df_aire['MAGNITUD'].map(aire_vars)

# Filtrar solo combinaciones con las 3 magnitudes por fecha
grupo_aire = df_aire.groupby(['ANO', 'MES', 'DIA', 'ESTACION', 'NOMBRE_ESTACION'])['VARIABLE'].nunique().reset_index()
grupo_aire = grupo_aire[grupo_aire['VARIABLE'] == 3]
df_aire = df_aire.merge(grupo_aire[['ANO', 'MES', 'DIA', 'ESTACION', 'NOMBRE_ESTACION']], on=['ANO', 'MES', 'DIA', 'ESTACION', 'NOMBRE_ESTACION'])

# Normalizar los valores por variable
df_aire['VALOR_ESCALADO'] = df_aire.groupby('VARIABLE')['VALOR'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Selectores interactivos mejorados
selector_ano = alt.selection_point(
    fields=['ANO'],
    bind=alt.binding_select(options=sorted(df_aire['ANO'].dropna().unique()), name="Año"),
    name='Año'
)
selector_mes = alt.selection_point(
    fields=['MES'],
    bind=alt.binding_select(options=sorted(df_aire['MES'].dropna().unique()), name="Mes"),
    name='Mes'
)
selector_dia = alt.selection_point(
    fields=['DIA'],
    bind=alt.binding_select(options=sorted(df_aire['DIA'].dropna().unique()), name="Día"),
    name='Día'
)
selector_estacion = alt.selection_point(
    fields=['NOMBRE_ESTACION'],
    bind=alt.binding_select(options=sorted(df_aire['NOMBRE_ESTACION'].unique()), name="Estación"),
    name='Estación'
)

# Gráfico radial con filtros separados
grafico_aire = alt.Chart(df_aire).transform_filter(selector_ano).transform_filter(selector_mes).transform_filter(selector_dia).transform_filter(selector_estacion).mark_arc(innerRadius=50).encode(
    theta=alt.Theta('VARIABLE:N', sort=list(aire_vars.values())),
    radius='VALOR_ESCALADO:Q',
    color='VARIABLE:N',
    tooltip=['VARIABLE:N', 'VALOR:Q', alt.Tooltip('VALOR_ESCALADO:Q', title='Escalado')]
).add_params(
    selector_ano, selector_mes, selector_dia, selector_estacion
).properties(
    width=400,
    height=400,
    title='Perfil diario de calidad del aire (normalizado)'
)






grafico_aire.save("grafico_aire.html")  # Exportación como HTML interactivo
grafico_aire


### Distribución acumulada de contaminantes por estación
Este gráfico de barras apiladas permite comparar la carga total medida de contaminantes por estación, distinguiendo cada tipo de contaminante mediante colores. Facilita una evaluación rápida de qué estaciones presentan mayor acumulación total.


In [ ]:
# Filtrar registros válidos
df_calidad_validas = df_calidad_largo[df_calidad_largo['VALIDO'] == 'V'].copy()


df_calidad_validas['MAGNITUD'] = df_calidad_validas['MAGNITUD'].map(diccionario_magnitudes).fillna(df_calidad_validas['MAGNITUD'].astype(str))

# Agrupar por nombre de estación
df_sum = (
    df_calidad_validas
    .groupby(['NOMBRE_ESTACION', 'MAGNITUD'])['VALOR']
    .sum()
    .reset_index()
)

# Gráfico de barras apiladas con nombres
chart_barras_apiladas = alt.Chart(df_sum).mark_bar().encode(
    x=alt.X('NOMBRE_ESTACION:N', title='Estación'),
    y=alt.Y('VALOR:Q', stack='zero', title='Valor acumulado'),
    color=alt.Color('MAGNITUD:N', title='Magnitud'),
    tooltip=['NOMBRE_ESTACION:N', 'MAGNITUD:N', 'VALOR:Q']
).properties(
    title='Contaminación total por estación y magnitud',
    width=800,
    height=400
)


chart_barras_apiladas.save("chart_barras_apiladas.html")  # Exportación como HTML interactivo
chart_barras_apiladas
